In [ ]:
# library imports and test set
import numpy as np
import tensorflow as tf
print(np.__version__)
print(tf.__version__)

def load_data():
    import tensorflow as tf
    print('Using tensorflow version {}.'.format(tf.__version__))
    (x_train, y_train), (x_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()
    x_train = x_train.astype('float32') / 255
    x_test = x_test.astype('float32') / 255
    # convert labels to categorical samples
    y_train = tf.keras.utils.to_categorical(y_train, num_classes=10)
    y_test = tf.keras.utils.to_categorical(y_test, num_classes=10)
    print('Loaded Fashion-MNIST into x_train, y_train, x_test, y_test.')
    print('Shapes: x_train: {}, y_train: {}, x_test: {}, y_test: {}'.format(x_train.shape, y_train.shape, x_test.shape, y_test.shape))
    return ((x_train, y_train), (x_test, y_test))

(x_train, y_train), (x_test, y_test) = load_data()

import tensorflow as tf
from tensorflow.keras import layers, models

def compile_and_fit(model, x_train, y_train, 
                    learning_rate=10e-3, verbosity=1, 
                    epochs=40, batch_size=256):
    
    # compile a model
    opt = tf.keras.optimizers.SGD(learning_rate=learning_rate, momentum=0.5)
    loss_func = tf.keras.losses.CategoricalCrossentropy()
    model.compile(optimizer=opt,
                  loss=loss_func,
                  metrics=['accuracy'])
    
    ### fit model
    history_dropout = model.fit(x=x_train, y=y_train,
                                epochs=epochs, batch_size=batch_size,
                                validation_data=(x_test, y_test),
                                verbose=verbosity)
    return history_dropout.history

In [ ]:
# custom resnet18 block
class Resnet18Block(tf.keras.Model):
    def __init__(self, kernel_size, filters, downscale):
        super(Resnet18Block, self).__init__()
        self.kernel_size = kernel_size
        self.filters = filters
        self.downscale=downscale
        
        if downscale is False:
            self.conv2a = tf.keras.layers.Conv2D(self.filters, self.kernel_size, padding='same')
            self.conv2b = tf.keras.layers.Conv2D(self.filters, self.kernel_size, padding='same')

        elif downscale is True:
            self.conv2a = tf.keras.layers.Conv2D(self.filters, self.kernel_size, strides = 2, padding='same')
            self.conv2b = tf.keras.layers.Conv2D(self.filters, self.kernel_size, padding='same')
            self.convshortcut = tf.keras.layers.Conv2D(self.filters, kernel_size=1, strides=2)
        
    def call(self, input_tensor, training=False):
        x = self.conv2a(input_tensor)
        x = tf.nn.relu(x)
        
        x = self.conv2b(x)
        
        if self.downscale is False:
            return tf.nn.relu(x + input_tensor)
        
        elif self.downscale is True:
            return tf.nn.relu(x + self.convshortcut(input_tensor))

In [ ]:
# example usage
resnet18 = tf.keras.models.Sequential()
initializer = tf.keras.initializers.HeNormal() # xaiming he initialisation
resnet18.add(tf.keras.layers.Reshape((28,28,1), input_shape=(28,28)))

resnet18.add(Resnet18Block(kernel_size=3, filters=64, downscale=False))
resnet18.add(Resnet18Block(kernel_size=3, filters=64, downscale=False))

resnet18.add(Resnet18Block(kernel_size=3, filters=128, downscale=True))
resnet18.add(Resnet18Block(kernel_size=3, filters=128, downscale=False))

resnet18.add(Resnet18Block(kernel_size=3, filters=256, downscale=True))
resnet18.add(Resnet18Block(kernel_size=3, filters=256, downscale=False))

resnet18.add(Resnet18Block(kernel_size=3, filters=512, downscale=True))
resnet18.add(Resnet18Block(kernel_size=3, filters=512, downscale=False))

resnet18.add(tf.keras.layers.GlobalAveragePooling2D())

resnet18.add(tf.keras.layers.Dense(10, activation=None, kernel_initializer=initializer, name="final_layer"))
resnet18.add(tf.keras.layers.Softmax())

resnet18._name='resnet_18'